In [ ]:
import random
from collections import Counter

import tqdm
import plotly.graph_objects as go

SUITS = ['hearts', 'diamonds', 'clubs', 'spades']
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', '1', 'J', 'Q', 'K', 'A']

def create_deck():
    return [(rank, suit) for suit in SUITS for rank in RANKS]

def rank_hand(hand):
    values = [RANKS.index(card[0]) for card in hand]
    suits = [card[1] for card in hand]
    
    value_counts = Counter(values)
    suit_counts = Counter(suits)
    
    is_flush = max(suit_counts.values()) >= 5
    sorted_values = sorted(values, reverse=True)
    
    is_straight = False
    for i in range(len(sorted_values) - 4):
        if sorted_values[i] - sorted_values[i + 4] == 4:
            is_straight = True
            break

    if is_flush and is_straight:
        return ('straight flush', sorted_values)
    elif 4 in value_counts.values():
        return ('four of a kind', sorted_values)
    elif 3 in value_counts.values() and 2 in value_counts.values():
        return ('full house', sorted_values)
    elif is_flush:
        return ('flush', sorted_values)
    elif is_straight:
        return ('straight', sorted_values)
    elif 3 in value_counts.values():
        return ('three of a kind', sorted_values)
    elif list(value_counts.values()).count(2) >= 2:
        return ('two pair', sorted_values)
    elif 2 in value_counts.values():
        return ('one pair', sorted_values)
    else:
        return ('high card', sorted_values)

def simulate_game(hand, num_opponents, hand_rankings):
    deck = create_deck()
    for card in hand:
        deck.remove(card)
    
    opponents_hands = [[] for _ in range(num_opponents)]
    
    for _ in range(2):  # Each player gets two cards, one per round
        for opponent_hand in opponents_hands:
            card = random.choice(deck)
            opponent_hand.append(card)
            deck.remove(card)
    
    community_cards = random.sample(deck, 5)
    
    all_hands = [list(hand) + community_cards] + [op_hand + community_cards for op_hand in opponents_hands]
    hand_ranks = [rank_hand(h) for h in all_hands]
    
    best_rank = max(hand_ranks, key=lambda x: (hand_rankings.index(x[0]), x[1]))
    if hand_ranks[0] == best_rank:
        return ('win', rank_hand(list(hand) + community_cards)[0])
    return ('lose', rank_hand(list(hand) + community_cards)[0])

def calculate_equity(hand, num_opponents, num_simulations=1000):
    hand_rankings = ['high card', 'one pair', 'two pair', 'three of a kind', 'straight', 'flush', 'full house', 'four of a kind', 'straight flush']
    win_counts = 0
    hand_type_counts = Counter()
    
    for _ in range(num_simulations):
        result, hand_type = simulate_game(hand, num_opponents, hand_rankings)
        if result == 'win':
            win_counts += 1
        hand_type_counts[hand_type] += 1
    
    equity = win_counts / num_simulations
    hand_type_probs = {hand_type: hand_type_counts[hand_type] / num_simulations for hand_type in hand_rankings}
    
    return equity, hand_type_probs

def calculate_odds_for_hand_pairs(num_opponents, num_simulations=1000):
    hand_pairs = []
    
    # Create all possible hand pairs (suited and non-suited)
    for i in range(len(RANKS)):
        for j in range(i, len(RANKS)):
            if i == j:  # Pair hands (e.g., AA)
                for suit1 in SUITS:
                    for suit2 in SUITS:
                        if suit1 != suit2:  # Non-suited pairs
                            hand_pairs.append([(RANKS[i], suit1), (RANKS[j], suit2)])
            else:  # Non-pair hands (e.g., AK)
                for suit1 in SUITS:
                    for suit2 in SUITS:
                        if suit1 != suit2:  # Non-suited
                            hand_pairs.append([(RANKS[i], suit1), (RANKS[j], suit2)])
                        else:  # Suited
                            hand_pairs.append([(RANKS[i], suit1), (RANKS[j], suit2)])
    
    odds = {}
    
    for hand in tqdm.tqdm(hand_pairs):
        equity, _ = calculate_equity(hand, num_opponents, num_simulations)
        hand_str = f"{hand[0][0]}{hand[1][0]}{'s' if hand[0][1] == hand[1][1] else 'o'}"
        odds[hand_str] = equity
    
    return odds

def plot_heatmap(odds):
    # Prepare data for the heatmap
    ranks = RANKS[::-1]
    heatmap_data = {rank: {rank2: 0 for rank2 in RANKS} for rank in ranks}
    
    for hand_str, equity in odds.items():
        rank1 = hand_str[0]
        rank2 = hand_str[1]
        suited = hand_str[2] == 's'
        
        if suited:
            heatmap_data[rank1][rank2] = equity * 100
        else:
            
            heatmap_data[rank2][rank1] = equity * 100
    
    x_labels = RANKS
    y_labels = ranks
    z_values = [[heatmap_data[rank1][rank2] for rank2 in x_labels] for rank1 in y_labels]
    
    fig = go.Figure(data=go.Heatmap(
        z=z_values,
        x=x_labels,
        y=y_labels,
        colorscale=['red', 'yello', 'green'],
    ))
    
    fig.update_layout(
        title='Poker Hand Equity Heatmap',
        xaxis_nticks=36
    )
    
    fig.show()

if __name__ == "__main__":
    hand = [('A', 'spades'), ('J', 'clubs')]  # Example hand
    num_opponents = 4  # Example number of opponents
    num_simulations = 50_000  # Number of simulations for better accuracy
    
    equity, hand_type_probs = calculate_equity(hand, num_opponents, num_simulations)
    
    print(f"Equity for hand {hand} against {num_opponents} opponents: {equity * 100:.2f}%")
    print("Hand type probabilities:")
    for hand_type, prob in hand_type_probs.items():
        print(f"{hand_type}: {prob * 100:.2f}%")
    
    odds = calculate_odds_for_hand_pairs(num_opponents, 40_000)
    print("Odds for hand pairs:")
    # for hand_str, equity in odds.items():
    #     print(f"{hand_str}: {equity * 100:.2f}%")
    
    plot_heatmap(odds)
